In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy import units as u
from pykoa.koa import Koa
from tqdm.auto import tqdm
import os

In [2]:
sample_file = 'Gr8stars_GaiaeDR3_formatted.csv'
hires_obs_file = 'koa_out/all.csv'
koa_dir = 'koa_out/'

In [3]:
sample = Table.read(sample_file)
hires_obs = Table.read(hires_obs_file)

#### generate the list of targets for KOA online query:

In [5]:
koa_upload_file = 'koa_upload_file.csv'
with open(koa_upload_file, 'w') as f:
    f.write('object,ra,dec\n')
    for s in sample:
        if s['spectrum_hires'] == 'true':
            f.write('{0},{1:.16f},{2:.16f}\n'.format(s['primary_name'], s['ra_j2000'], s['dec_j2000']))

#### set up subdirectories for the data:

In [ ]:
for s in sample:
    if s['spectrum_hires'] == 'true':
        object_obs = Table.read(koa_dir+'Gaia_EDR3_{0}.tbl'.format(s['gaiaedr3_source_id']), format='ascii')
        good_mask = (object_obs['iodout']=='T') & (object_obs['specres'] >= 60000) & (object_obs['imagetyp'] == 'object')
        good_object_obs = object_obs[good_mask]
        
        object_dir = './hires/{0}'.format(name)
        metatable = object_dir+'/obs.csv'
        
        try:  # make the directory
            os.mkdir('./hires/{0}'.format(name))
        except FileExistsError:
            pass
        good_object_obs.write(metatable, overwrite=True) # populate it with names of observations to be copied over

#### copy the relevant data over into each subdirectory: